In [1]:
import datetime
import numpy as np
import pandas as pd
import pickle as pkl
import tensorflow as tf
import time

/home/ecbm6040-as5281/miniconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
truncate_len = 800
batch_size = 32

In [3]:
action_movie_lover = """i really love action movies they are my favorite kind of movie because i love to watch the good guys
win. some of my favorite actors are jet li and jackie chan because they were the last great actors who
actually knew how to fight. modern action movie actors are just pretty faces and the editors swap camera
angles when supposed hits make contact"""
action_movie_hater = """I really hate action movies. jackie chan and jet li are the worst. their old fashioned
special effects, if you can even call them that, are out dated and boring. why are people even still paying
for them to make movies"""
item = """this movie is great because of the way the strong, clever, hero saves the day at the end. as always
jackie chans action directing and stunts are amazing, i'm so glad he doesn't use a stunt double liek
some other actors that don't need to be mentioned"""


In [4]:
class Batch_Dataset(object):
    def __init__(self, user_review_location, item_review_location, rating_location, pad_length, pad_value, batch_size):
        user_review_list = [line.split()[:truncate_len] for line in self._get_lines(user_review_location)]
        self.user_review_list = self._pad_if_necessary(user_review_list, pad_value, pad_length)
        
        item_review_list = [line.split()[:truncate_len] for line in self._get_lines(item_review_location)]
        self.item_review_list = self._pad_if_necessary(item_review_list, pad_value, pad_length)
        
        self.ratings = [float(rating) for rating in self._get_lines(rating_location)]
        
        self.current = 0
        self.batch_size = batch_size
        self.stop_len = len(self.ratings)
    
    def _get_lines(self, fname):
        with open(fname, "rt") as data:
            return data.read().splitlines()
        
    def _pad_if_necessary(self, list_of_lists, pad_value, pad_length):
        return [np.array(line + [pad_value] * (pad_length - len(line))) for line in list_of_lists]
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current + self.batch_size > self.stop_len:
            self.current = 0
            self.prev = 0
            raise StopIteration
        else:
            self.prev = self.current
            self.current += self.batch_size
            return (np.array(self.user_review_list[self.prev: self.current]),
                    np.array(self.item_review_list[self.prev: self.current]),
                    np.array(self.ratings[self.prev: self.current]).reshape(self.batch_size, 1))

In [5]:
emb_size = 50
filters = 10
kernel_size = 3
n_epochs = 824

s = time.time()

tf.reset_default_graph()

with open("data/dictionary.pkl", "rb") as f:
    dictionary = pkl.load(f)

values = list(range(len(dictionary)))
keys = list(dictionary)

table = tf.contrib.lookup.HashTable(
  tf.contrib.lookup.KeyValueTensorInitializer(keys, values), -1
)

word_embeddings = tf.get_variable(
    "word_embeddings",
    shape=[len(dictionary), emb_size]
)

u_inputs = tf.placeholder(tf.string, (batch_size, truncate_len), name="user_inputs")
i_inputs = tf.placeholder(tf.string, (batch_size, truncate_len), name="item_inputs")
ratings_input = tf.placeholder(tf.float64, (batch_size, 1), name="ratings")

u_inputs_indices = table.lookup(u_inputs)
i_inputs_indices = table.lookup(i_inputs)

u_inputs_embedded = tf.nn.embedding_lookup(word_embeddings, u_inputs_indices)
i_inputs_embedded = tf.nn.embedding_lookup(word_embeddings, i_inputs_indices)

user_conv1 = tf.layers.conv1d(
    u_inputs_embedded,
    filters,
    kernel_size,
    use_bias=True,
    activation=tf.nn.tanh,
    name="user_conv")

item_conv1 = tf.layers.conv1d(
    i_inputs_embedded,
    filters,
    kernel_size,
    use_bias=True,
    activation=tf.nn.tanh,
    name="item_conv")

user_max_pool1 = tf.layers.max_pooling1d(user_conv1, 2, 1)
item_max_pool1 = tf.layers.max_pooling1d(item_conv1, 2, 1)

user_flat = tf.layers.flatten(user_max_pool1)
item_flat = tf.layers.flatten(item_max_pool1)

user_dense = tf.layers.dense(user_flat, 64, activation=tf.nn.relu)
item_dense = tf.layers.dense(item_flat, 64, activation=tf.nn.relu)

predictions = tf.reduce_sum( tf.multiply( user_dense, item_dense ), 1, keep_dims=True )

loss = tf.losses.mean_squared_error(ratings_input, predictions)

optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)

train_op = optimizer.minimize(
    loss=loss)

vars_init = tf.global_variables_initializer()
tables_init = tf.tables_initializer()


with tf.Session() as sess:
    
    sess.run(vars_init)
    sess.run(tables_init)
    
    dataset = Batch_Dataset("data/train_u_reviews.txt",
                            "data/train_i_reviews.txt",
                            "data/train_ratings.txt",
                            truncate_len,
                            "unk",
                            batch_size)
    
    i = 0
    for user_batch, item_batch, rating_batch in dataset:
        
        i += 1
        
        _, l = sess.run([train_op, loss], feed_dict={
            u_inputs: user_batch,
            i_inputs: item_batch,
            ratings_input: rating_batch
        })
        if i % 101 == 0:
            print("{}: epoch {}, loss {:.2f}".format(str(datetime.datetime.now()), i, l))
            
e = time.time()

print("ran {} seconds".format(e - s))

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
2018-04-15 22:54:31.248233: epoch 101, loss 1.49
2018-04-15 22:54:34.302774: epoch 202, loss 2.01
2018-04-15 22:54:37.369570: epoch 303, loss 0.66
2018-04-15 22:54:40.413698: epoch 404, loss 1.53
2018-04-15 22:54:43.445498: epoch 505, loss 0.97
2018-04-15 22:54:46.480845: epoch 606, loss 1.30
2018-04-15 22:54:49.541625: epoch 707, loss 0.89
2018-04-15 22:54:52.621426: epoch 808, loss 1.02
ran 45.2598602771759 seconds
